In [2]:
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/data/ship.csv

--2021-01-21 12:38:46--  https://github.com/mlittmancs/great_courses_ml/raw/master/data/ship.csv
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/data/ship.csv [following]
--2021-01-21 12:38:47--  https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/data/ship.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.40.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.40.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61194 (60K) [text/plain]
Saving to: ‘ship.csv’

ship.csv            100%[===================>]  59.76K  --.-KB/s    in 0.05s   

2021-01-21 12:38:48 (1.26 MB/s) - ‘ship.csv’ saved [61194/61194]



In [3]:
import csv

first = True
with open('ship.csv') as f:
    csvdata = csv.reader(f, delimiter=',')
    data = []
    for row in csvdata:
        if not first: data += [row]
        first = False

array = []
for col in range(len(data[0])):
    array += [{}]
    new = 0
    for i in range(len(data)):
        line = data[i]
        if line[col] not in array[col]:
            array[col][line[col]] = new
            new += 1

alldata = []
alllabs = []
for line in data:
    alllabs += [int(line[1])]
    if line[5] == '': line[5] = '50'
    alldata += [[int(line[2]), array[4][line[4]], float(line[5]), int(line[6]), int(line[7]), float(line[9]), line[11]=='S', line[11]=='C', line[11]=='Q']]
feats = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked S','Embarked C','Embarked Q']

In [4]:
import random

trainmask = [random.randint(0,2) for i in range(len(alldata))]

traindata   = [alldata[i] for i in range(len(alldata)) if trainmask[i]<2]
trainlabs   = [alllabs[i] for i in range(len(alldata)) if trainmask[i]<2]
testdata    = [alldata[i] for i in range(len(alldata)) if trainmask[i]==2]
testlabs    = [alllabs[i] for i in range(len(alldata)) if trainmask[i]==2]

In [5]:
from sklearn.neural_network import MLPClassifier

nhidden = 60
clf = MLPClassifier(hidden_layer_sizes=[nhidden],max_iter=50000)
clf = clf.fit(traindata,trainlabs)
pred = clf.predict(testdata)
[sum([pred[i] != testlabs[i] for i in range(len(testlabs))]) / len(testlabs)]

[0.21710526315789475]

In [6]:
# feats = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked S','Embarked C','Embarked Q']

imp = []
for v in alldata:
    real = v[1]
    v[1] = 0
    asmale = clf.predict_proba([v])[0][1]
    v[1] = 1
    asfemale = clf.predict_proba([v])[0][1]
    v[1] = real
    imp += [asfemale-asmale]

print(sum(imp)/len(imp))

0.46844429683301314


In [7]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=500)
clf.fit(traindata,trainlabs)

pred = clf.predict(traindata)
trainerr = sum([pred[i] != trainlabs[i] for i in range(len(trainlabs))]) / len(trainlabs)
pred = clf.predict(testdata)
testerr = sum([pred[i] != testlabs[i] for i in range(len(testlabs))]) / len(testlabs)

print(trainerr, testerr)

0.17546848381601363 0.23026315789473684


### Logistic Regression:
A one-layered NN equivalent to sigmoid activation
* one weight per input feature
* output activations computed by transforming weighted sum with sigmoid function

In [8]:
for i in range(len(feats)):
    print(feats[i],clf.coef_[0][i])

Pclass -0.9376646133933938
Sex 2.6501970315282786
Age -0.026270890610032602
SibSp -0.2734282576527669
Parch -0.004343020664387434
Fare 0.0020327021766925117
Embarked S -0.5039976276625564
Embarked C 0.15301992315134014
Embarked Q 0.1613244985004878


### Proxy Measure:
a variable that's closely tied to the desired variable (ie: zipcode and income) which can sometimes
be used as a substitute in certain measurements.